# Airlane Passengers Load Prediction - RAMP


![title](cover.png)
<br>

## Table of contents
[1. Context and objective](#section1)

[2. Exploring the datasets (EDA)](#section2)

* [A. Structure of the datasets](##section2A)
* [B. Data cleaning and preprocessing](##section2B)


[3. Prediction](#section3)
* [A. Preparing the training dataset](##section3A)
* [B. Selecting and hyper tuning the models](##section3B)

[4. Conclusion and next steps](#section3)

<br>

<a id="Section1"></a>
## 1. Context and objective

This notebook is our final project for the class [MAP536P - Python for Data Science](https://moodle.polytechnique.fr/course/view.php?id=6124).

The objective is to **forecast airplane passengers load** in the United States. 

## 2. Dataset Exploration and Feature Engineering

In [2]:
%matplotlib inline
import imp
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
pd.set_option('display.max_columns', None)

### Initial Datasets

In [3]:
problem = imp.load_source('', 'problem.py') #problem is a module with different methods

In [4]:


X_df, y_array = problem.get_train_data() #get_train_data() is one of those methods
X_train = X_df.copy()
X_train["passengers"] = y_array
X_train.head()

,DateOfDeparture,Departure,Arrival,WeeksToDeparture,std_wtd,passengers
0,2012-06-19,ORD,DFW,12.875000,9.812647,12.331296
1,2012-09-10,LAS,DEN,14.285714,9.466734,10.775182
2,2012-10-05,DEN,LAX,10.863636,9.035883,11.083177
3,2011-10-09,ATL,ORD,11.480000,7.990202,11.169268
4,2012-02-21,DEN,SFO,11.450000,9.517159,11.269364
